In [112]:
import earthshadow

The radius is calculated for an observer at the center of the Earth. 
This is the easiest way to check if an object is in the shadow, see `dist_from_shadow_center` below. 

In [113]:
earthshadow.get_shadow_radius(orbit='leo')

<Quantity 72.75179437 deg>

The shadow is much smaller when it intersects the higher geosynchronous orbit:

In [114]:
earthshadow.get_shadow_radius(orbit='geo')

<Quantity 8.69070548 deg>

Now we can check the object's distance from the center of the shadow. 
This depends on the position of the observer and time of day. 
The observer time/position affects how a coordinate (RA/Dec, which is a point at infinity) 
is translated to a point intersecting the orbit. 
Note that the resulting answer is returned as an angle measured from the center of the earth, 
    so it can be compared with the output of `get_shadow_radius` above.

In [115]:
from astropy.coordinates import EarthLocation
obs=EarthLocation.of_site('Palomar')
obs.to_geodetic()

GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)

Local time at Palomar is about an hour after sunset.

In [116]:
from astropy.time import Time
import pytz, datetime

ra = 221.9927447
dec= 0.9557557

jd= 2458665.6915278
t = Time(jd, format='jd', location=obs)
dt = t.datetime.replace(tzinfo=datetime.timezone.utc)

print(
        f'UTC time: {dt.astimezone(pytz.timezone("UTC"))}, '
        f'local time: {dt.astimezone(pytz.timezone("US/Pacific"))}'
    )


UTC time: 2019-07-01 04:35:48.001935+00:00, local time: 2019-06-30 21:35:48.001935-07:00


Get the sun's position and the anti-sun point's position: 

In [117]:
from astropy.coordinates import get_sun
sun = get_sun(t)
anti_sun = earthshadow.get_anti_sun(jd)
print(f'Sun: {sun.ra:.1f}, {sun.dec:.1f} | anti sun: {anti_sun.ra:.1f}, {anti_sun.dec:.1f}')

Sun: 99.6 deg, 23.1 deg | anti sun: 279.6 deg, -23.1 deg


Target is close to the prime meridian: 

In [118]:
lst = t.sidereal_time("mean").value*15
print(f'LST: {lst:.1f} deg, RA= {ra:.1f} deg')

LST: 231.0 deg, RA= 222.0 deg


Because we are looking at LEO, the part of the sky we are observing is always going 
to be close to where we are on Earth (RA->LST, Dec->lat). 
Since the observatory (LST, lat) is (231, 33) and the anti sun is (280, -23), the expected delta is just: 

In [119]:
obs_ra=obs.get_gcrs(t).ra
obs_dec=obs.get_gcrs(t).dec

In [120]:
import numpy as np
np.sqrt((obs_ra-anti_sun.ra)**2 + (obs_dec-anti_sun.dec)**2)

<Quantity 74.54569878 deg>

Which is close to what we get using the calculator. 

In [121]:
earthshadow.dist_from_shadow_center(ra=ra, dec=dec, time=jd, orbit='leo', obs=obs)

<Angle [72.28888326] deg>

This is outside the shadow, because the radius is 72 degrees, and we need to also include about 2 degrees for atmospheric refraction (which shrinks the shadow). 

If we observe the same coordinates but ask about GEO orbit (where there is much lower parallax), 
we should use the coordinates of the target, not the observatory's LST/lat.

In [122]:
np.sqrt((ra-anti_sun.ra.value)**2 + (dec-anti_sun.dec.value)**2)

62.4449908914747

Indeed this is already much closer to the exact number from the calculator. 

In [123]:
earthshadow.dist_from_shadow_center(ra=ra, dec=dec, time=jd, orbit='geo', obs=obs)

<Angle [62.16027266] deg>

If we were observing a field close to the prime meridian around midnight, it would be closer to the center of the shadow:

In [124]:
from astropy.time import Time
import datetime
import pytz
jd=2458665.6915278 + 0.1 + 1/24  # add a few hours to get to midnight (1am for daylight saving)! 
t = Time(jd, format='jd', location=obs)
dt = t.datetime.replace(tzinfo=datetime.timezone.utc)
print(
        f'UTC time: {dt.astimezone(pytz.timezone("UTC"))}, '
        f'local time: {dt.astimezone(pytz.timezone("US/Pacific"))}'
    )


UTC time: 2019-07-01 07:59:48.001929+00:00, local time: 2019-07-01 00:59:48.001929-07:00


This time the observatory's LST should be closer to the anti-sun's RA=280

In [125]:
obs_ra=obs.get_gcrs(t).ra
obs_dec=obs.get_gcrs(t).dec
print(f'observatory coordinates: {obs_ra:.1f}, {obs_dec:.1f}')

observatory coordinates: 282.0 deg, 33.2 deg


The rough estimate of the angle difference would still be quite large, because of the declination difference:

In [126]:
np.sqrt((obs_ra-anti_sun.ra)**2+(obs_dec-anti_sun.dec)**2)

<Quantity 56.35044123 deg>

In [127]:
earthshadow.dist_from_shadow_center(ra=ra, dec=dec, time=jd, orbit='leo', obs=obs)

<Angle [54.83570716] deg>

If we observe GEO the actual coordinates we put in will be much closer to the coordinates. We can specify a new target that is on the prime meridian and on the ecliptic (declination of the anti solar point):

The distance from the shadow's center should be a few degrees. 
It is really important to consider the parallax here, since Palomar is at +33 and the anti-solar point is at -23

In [128]:
np.sqrt((ra - anti_sun.ra.value)**2 + (dec-anti_sun.dec.value)**2)

62.4449908914747

In [129]:
ra=obs_ra.value
dec=anti_sun.dec.value
earthshadow.dist_from_shadow_center(ra=ra, dec=dec, time=jd, orbit='geo', obs=obs)

<Angle [7.51990936] deg>

If we ask for a much higher orbit the distance should converge to the naive calculation:

In [130]:
earthshadow.dist_from_shadow_center(ra=ra, dec=dec, time=jd, orbit=1e6, obs=obs)

<Angle [2.09113313] deg>